In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="I/O Code"></form>''')

# This is a test for both Jupyter Notebook and my first quant algorithm

### Kai Guo 
### 2018/03/15

In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import tushare as ts
import bt
import matplotlib.pyplot as plt

# set parameters
plt.style.use('bmh')
start_date = "2010-01-01"
concept = "婴童"
code_refer = ["000001"]
# set parameters end

df = ts.get_concept_classified()
df = df[df.c_name.str.contains(concept)]
code_list = df.code.tolist()


# replace bt.get()
def get_data(tickers, start):
    if len(tickers) == 1:
        for i in tickers:
            data = ts.get_k_data(i, start=start)
            data.date = pd.to_datetime(data.date)
            data.set_index("date", inplace=True)
            data = pd.DataFrame(data.loc[:, "close"])
            data.columns = tickers
            return data
    else:
        data = pd.DataFrame()
        for i in tickers:
            temp = ts.get_k_data(i, start=start)
            temp.date = pd.to_datetime(temp.date)
            temp.set_index("date", inplace=True)
            temp = temp.loc[:, "close"]
            data = pd.concat([data, temp], axis=1)
        data.columns = tickers
        data = data.fillna(method='ffill')
        return data


def above_sma(tickers, sma_per, start='2010-01-01', name='above_sma'):
    """
    Long securities that are above their n period
    Simple Moving Averages with equal weights.
    """
    data = get_data(tickers=tickers, start=start)

    # calc sma
    sma = data.rolling(sma_per).mean()

    # create strategy
    s = bt.Strategy(name, [bt.algos.SelectWhere(data > sma),
                           bt.algos.WeighEqually(),
                           bt.algos.Rebalance()])

    # now we create the backtest
    return bt.Backtest(s, data)


# simple backtest to test long-only allocation
def long_only_ew(tickers, start='2010-01-01', name='long_only_ew'):
    s = bt.Strategy(name, [bt.algos.RunOnce(),
                           bt.algos.SelectAll(),
                           bt.algos.WeighEqually(),
                           bt.algos.Rebalance()])

    data = get_data(tickers=tickers, start=start)

    return bt.Backtest(s, data)



In [ ]:
# portfolio sma?
benchmark = long_only_ew(tickers=code_refer, start=start_date, name='benchmark')
sma10 = above_sma(tickers=code_list, sma_per=10, name='sma10', start=start_date)
sma20 = above_sma(tickers=code_list, sma_per=20, name='sma20', start=start_date)
sma40 = above_sma(tickers=code_list, sma_per=40, name='sma40', start=start_date)
sma100 = above_sma(tickers=code_list, sma_per=100, name='sma100', start=start_date)
sma200 = above_sma(tickers=code_list, sma_per=200, name='sma200', start=start_date)
res = bt.run(sma10, sma20, sma40, sma100, sma200, benchmark)
print("\n")
print(res.display())


timed out


In [ ]:
import numpy as np
res.plot()
plt.show()

# ===================================TBD
# ma crossover
data1 = get_data(['002083'], "2010-01-01")

sma50 = data1.rolling(50).mean()
sma200 = data1.rolling(200).mean()
# now we need to calculate our target weight DataFrame
# first we will copy the sma200 DataFrame since our weights will have the same strucutre
tw = sma200.copy()
# set appropriate target weights
tw[sma50 > sma200] = 1.0
tw[sma50 <= sma200] = -1.0
# here we will set the weight to 0 - this is because the sma200 needs 200 data points before
# calculating its first point. Therefore, it will start with a bunch of nulls (NaNs).
tw[sma200.isnull()] = np.nan

tmp = bt.merge(tw, data1, sma50, sma200)
tmp.columns = ['tw', 'price', 'sma50', 'sma200']
ax = tmp.plot(figsize=(15, 5), secondary_y=['tw'])
plt.show()

ma_cross = bt.Strategy('ma_cross', [bt.algos.WeighTarget(tw),
                                    bt.algos.Rebalance()])

t = bt.Backtest(ma_cross, data1)
res1 = bt.run(t)
res1.plot()
plt.show()



In [ ]:
res.plot_correlation()
plt.show()

In [ ]:
res.plot_histogram()